## KoNLPy 형태소 분석기 선정

### 대상
1. hannanum
2. kkma
3. okt

### 선정 방법

책 10권에 대해 각각 10개의 키워드 선정
hannanum,kkma,okt 명사추출 기능으로 해당 키워드가 구분되는지 체크 

### 도서목록
1. 리액트를 다루는 기술 = The art of React : 실무에서 알아야 할 기술은 따로 있다!

2. (Do it!) 장고 + 부트스트랩 파이썬 웹 개발의 정석 : 만들면서 배우는 웹 개발 A to Z
3. 머신러닝 디자인 패턴 : 효율적인 머신러닝 파이프라인과 MLOps를 구축하는 30가지 디자인 패턴
4. 핸즈온 머신러닝
5. 파이썬을 이용한 데이터 분석의 정석 : 넘파이, 판다스, 맷플롯립과 실전 예제로 배우는
6. 스파크 완벽 가이드 : 스파크를 활용한 빅데이터 처리와 분석의 모든 것
7. 처음 시작하는 딥러닝
8. 컴퓨터 사이언스 부트캠프 with 파이썬
9. 케라스 창시자에게 배우는 딥러닝
10. 처음 배우는 리액트 네이티브 크로스 플랫폼 앱 개발을 위한 실전 입문서


In [6]:
from konlpy.tag import Hannanum,Okt,Kkma
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec,KeyedVectors
import re

### 성능 비교에 사용 될 도서 10권 추출

In [3]:
bookinfo = pd.read_csv('./data/bookInfo12.csv',encoding='cp949')
testBooks = [9791160508796,9791163032069,9791162244845,9791162242964,9791186710708,9791162241288,9791162243343,9791160504095,9791160505979,9791162243879]
books = bookinfo[bookinfo['col1'].isin(testBooks)]
han = Hannanum()


### TF-IDF

In [31]:
list1 = books.iloc[3].tolist()
def TFIDF(wordList) :
    from operator import itemgetter

    str_list = [re.sub('\d','',str(a)) for a in wordList]
    str_list = list(filter(None, str_list))

    han = Hannanum()
    word = ' '.join(str_list)
    konlpword = han.nouns(word)
    konlpword = ' '.join(konlpword)

    vectorizer = TfidfVectorizer()
    sp_matrix = vectorizer.fit_transform([konlpword])

    # a,b
    a = np.array(vectorizer.get_feature_names_out())
    b = sp_matrix.toarray()[0]
    unique, counts = np.unique(b, return_counts=True)

    ### Top 10 추출
    k = 0
    for num in range(1,100) :
        if k < 10 :
            k += counts[-num]
        else :
            break

    result =[(a,b) for a,b in zip(a,b) if b >= unique[-num]]            
    result = sorted(result,key=itemgetter(1),reverse=True)
    return result

k = TFIDF(list1)
TFIDFKeyword = list(map(lambda x : x[0],k))

### Word2Vec

In [64]:
def tokenizer(item:pd.Series) :
    wordList = item.astype(str).tolist()
    str_list = [re.sub('\d','',str(a)) for a in wordList]
    str_list = list(filter(None, str_list))
    result = ' '.join(str_list)
    return result

wordsList=[tokenizer(i[1]) for i in bookinfo.iterrows()]
print('Complete wordsList Load!!')
konlpyWords = list(map(lambda x : han.nouns(x),wordsList ))
embedding_model = Word2Vec(sentences=konlpyWords, window = 2, min_count=50, workers=7, sg=1)

In [156]:
embedding_model.wv.similar_by_word('맷플롯립')

In [8]:
# embedding_model.wv.save_word2vec_format('booksTest1') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("booksTest1") # 모델 로드

In [83]:
keywordSearching = '임베딩'
result :tuple = loaded_model.most_similar(keywordSearching,topn=20)
keywordsWord2Vec = list(map(lambda x : x[0], result))
keywordsWord2Vec.append(keywordSearching)

In [88]:
import ast
kkk = pd.read_csv('./data/bookInfoKeyword.csv',encoding='cp949')
lists = kkk['keyword'].tolist()
def changeStringToList(strList) :
    strList = strList.replace(' ',', ')
    result = np.array(ast.literal_eval(strList))
    
    return ast.literal_eval(strList)

result = list(map(changeStringToList,lists))

# embedding_model = Word2Vec(sentences=result, window = 2, min_count=50, workers=7, sg=0)


In [66]:
# k = pd.DataFrame(result) # 6462개 키워드
# kbb= np.isin(k.values,keywordsWord2Vec)
# filteredIndex = k[kbb].index.tolist()

In [89]:
def findOverlapNum(result:list):
    return sum(np.in1d(result,keywordsWord2Vec))

sums = list(map(findOverlapNum,result))

In [95]:
testkkk = kkk.copy()
testkkk['sum'] = sums

a = testkkk[testkkk['sum'] > 1].sort_values(by='sum',ascending=False).drop_duplicates(subset='ISBN')

a



,Unnamed: 0,도서명,저자,출판사,ISBN,주제분류번호,등록일자,이미지주소,keyword,sum
4065,4065,비전 시스템을 위한 딥러닝 - 신경망을 활용한 엔드투엔드 컴퓨터 비전 애플리케이션 ...,"모하메드 엘겐디 (지은이), 심효섭 (옮긴이)",한빛미디어,9791162244975,004.73,2022-01-28,http://image.kyobobook.co.kr/images/book/large...,['비전' '컴퓨터' '딥러닝' '이미지' '신경망' '알고리즘' '프로젝트' '분...,3
1031,1031,"구글 BERT의 정석 :인공지능, 자연어 처리를 위한 BERT의 모든 것","수다르산 라비찬디란 지음;전희원,정승환,김형준 옮김",한빛미디어,9791162244852,004.735,2022-04-11,http://image.kyobobook.co.kr/images/book/large...,['모델' '언어' '이해' '학습' '처리' '트랜스포머' '보충' '자연어' '...,3
96,96,(BERT와 GPT로 배우는) 자연어 처리 :트랜스포머 핵심 원리와 허깅페이스 패키...,이기창 지음,이지스퍼블리싱,9791163033165,004.73,2022-02-07,http://image.kyobobook.co.kr/images/book/large...,['처리' '자연어' '모델' '실습' '학습' '문장' '최신' '언어' '핵심'...,2
1841,1841,텐서플로와 머신러닝으로 시작하는 자연어 처리(데이터 사이언스 시리즈 30),전창욱,위키북스,9791158391379,004.73,2019-03-29,http://image.kyobobook.co.kr/images/book/large...,['처리' '자연어' '모델' '데이터' '텍스트' '분류' '소개' '기반' '문...,2
4413,4413,파이썬으로 배우는 응용 텍스트 분석 (언어 인식 데이터 제품 개발을 위한 머신러닝),벤자민 벵포트|레베카 빌브로|토니 오제다,Jpub(제이펍),9791188621774,004.735,2021-01-19,http://image.kyobobook.co.kr/images/book/large...,['텍스트' '모델' '데이터' '언어' '분석' '말뭉치' '결론' '분류' '특...,2
4330,4330,(텐서플로 2와 머신러닝으로 시작하는) 자연어 처리 :로지스틱 회귀부터 BERT와 ...,"전창욱,최태균,조중현,신성진 지음",위키북스,9791158393168,004.73,2022-06-09,http://image.kyobobook.co.kr/images/book/large...,['모델' '자연어' '처리' '활용' '텍스트' '분류' '데이터' '한국어' '...,2
4311,4311,"케라스로 구현하는 딥러닝과 강화학습 :신경망 기초부터 CNN, RNN, GAN, 단...","안토니오 걸리,수짓 팔 지음 ;김창엽 옮김",에이콘,9791161750743,004.73,2018-02-10,http://image.kyobobook.co.kr/images/book/large...,['케라스' '신경망' '네트워크' '딥러닝' '활용' '학습' '예제' '설치' ...,2
4193,4193,자연어 처리와 컴퓨터 언어학 - 파이썬으로 개발하는 자연어 처리 서비스,"바르가브 스리니바사 디지칸 (지은이), 마창수 (옮긴이)",에이콘출판,9791161753140,004.735,2019-07-22,http://image.kyobobook.co.kr/images/book/large...,['텍스트' '분석' '스파이시' '데이터' '이해' '이용' '알고리즘' '파이썬...,2
3756,3756,자연어처리 바이블 (핵심이론.응용시스템.딥러닝),임희석,휴먼싸이언스,9791189057145,004.735,2020-03-13,http://image.kyobobook.co.kr/images/book/large...,['자연어처리' '참고문헌' '딥러닝' '기반' '분석' '기술' '모델' '학습'...,2
3169,3169,딥러닝 파이토치 교과서,서지영,길벗,9791165218942,005.13,2022-03-14,http://image.kyobobook.co.kr/images/book/large...,['딥러닝' '신경망' '파이토치' '학습' '구현' '알고리즘' '모델' '성능'...,2


In [70]:
BM = kkk.index.isin(filteredIndex)
kkk[BM]

,Unnamed: 0,도서명,저자,출판사,ISBN,주제분류번호,등록일자,이미지주소,keyword
94,94,"고전 컴퓨터 알고리즘 인 파이썬 :피보나치 수열부터 보드게임까지, 알고리즘으로 풀어...",데이비드 코펙 지음 ;최길우 옮김,한빛미디어,9791162242469,005.1,2022-02-07,http://image.kyobobook.co.kr/images/book/large...,['문제' '알고리즘' '연습문제' '적용사례' '컴퓨터' '신경망' '군집화' '...
96,96,(BERT와 GPT로 배우는) 자연어 처리 :트랜스포머 핵심 원리와 허깅페이스 패키...,이기창 지음,이지스퍼블리싱,9791163033165,004.73,2022-02-07,http://image.kyobobook.co.kr/images/book/large...,['처리' '자연어' '모델' '실습' '학습' '문장' '최신' '언어' '핵심'...
103,103,구글 BERT의 정석,수다르산 라비찬디란,한빛미디어,9791162244852,004.73,2021-11-16,http://image.kyobobook.co.kr/images/book/large...,['모델' '언어' '이해' '학습' '처리' '트랜스포머' '보충' '자연어' '...
127,127,"딥러닝 텐서플로 교과서 :기초부터 CNN, RNN, 시계열 분석, 성능 최적화, 자...",서지영 지음,길벗,9791165215477,004.73,2021-10-13,http://image.kyobobook.co.kr/images/book/large...,['딥러닝' '알고리즘' '학습' '신경망' '구현' '모델' '텐서플' '합성곱'...
276,276,혼자 공부하는 머신러닝+딥러닝,박해선,한빛미디어,9791162243664,004.73,2021-02-18,http://image.kyobobook.co.kr/images/book/large...,['핵심' '신경망' '문제' '머신러닝' '학습' '키워드' '확인' '과정' '...
...,...,...,...,...,...,...,...,...,...
5873,5873,"자연어 처리를 위한 트랜스포머 - Python, PyTorch, TensorFlow...","Denis Rothman (지은이), 유현중 (옮긴이)",DK로드북스,9791196965679,004.73,2022-04-18,http://image.kyobobook.co.kr/images/book/large...,['트랜스포머' '모델' '훈련' '요약' '아키텍처' '질문' '작업' '데이터셋...
5924,5924,데이터 과학 :입문과 토픽=Data Science :Introduction and ...,허명회 지음,자유아카데미,9791158081898,005.7,2018-06-08,http://image.kyobobook.co.kr/images/book/large...,['데이터' '과학' '학습' '입문' '공부' '토픽' '경우' '학습서' '필요...
5990,5990,손에 잡히는 R 프로그래밍,이준용,한빛미디어,9788968481642,005.54,2019-10-12,http://image.kyobobook.co.kr/images/book/large...,['프로그래밍' '데이터' '프로젝트' '코드' '함수' '작성' '사람' '패키지...
5997,5997,하둡과 스파크를 활용한 실용 데이터 과학,"오퍼 멘델리비치, 케이시 스텔라, 더글라스 에드라인 (지은이), 이춘오 (옮긴이)",길벗,9791160502213,005.74,2019-08-20,http://image.kyobobook.co.kr/images/book/large...,['데이터' '하둡' '과학' '활용' '스파크' '모델' '분석' '요약' '머신...


In [42]:
result2d = np.reshape(result,(-1,2))

result2d.shape

(3231, 2)

In [19]:
# kkk[kkk.keyword.str.contains('임베딩')]

,Unnamed: 0,도서명,저자,출판사,ISBN,주제분류번호,등록일자,이미지주소,keyword
37,37,파이썬 텍스트 마이닝 완벽 가이드,"박상언,강주영,정석찬 지음",위키북스,9791158393007,005.133,2022-04-11,http://image.kyobobook.co.kr/images/book/large...,['이용' '분류' '딥러닝' '이해' '텍스트' '분석' '기반' '토픽' '기법...
103,103,구글 BERT의 정석,수다르산 라비찬디란,한빛미디어,9791162244852,004.73,2021-11-16,http://image.kyobobook.co.kr/images/book/large...,['모델' '언어' '이해' '학습' '처리' '트랜스포머' '보충' '자연어' '...
127,127,"딥러닝 텐서플로 교과서 :기초부터 CNN, RNN, 시계열 분석, 성능 최적화, 자...",서지영 지음,길벗,9791165215477,004.73,2021-10-13,http://image.kyobobook.co.kr/images/book/large...,['딥러닝' '알고리즘' '학습' '신경망' '구현' '모델' '텐서플' '합성곱'...
334,334,한국어 임베딩,이기창,에이콘,9791161753508,004.735,2020-05-18,http://image.kyobobook.co.kr/images/book/large...,['임베딩' '모델' '자연어' '학습' '단어' '한국어' '수준' '처리' '문...
645,645,모두의 라즈베리 파이 with 파이썬 =Raspberry Pi for everyon...,"이시이 모루나,에사키 노리히데 지음 ;서수환 옮김",길벗,9791160507027,005.133,2019-03-08,http://image.kyobobook.co.kr/images/book/large...,['신경망' '훈련' '딥러닝' '모델' '케라스' '문제' '계층' '심층' '아...
1014,1014,파이썬 텍스트 마이닝 완벽 가이드 :자연어 처리 기초부터 딥러닝 기반 BERT 모델까지,"박상언, 강주영, 정석찬 지음",위키북스,9791158393007,005.133,2022-04-22,http://image.kyobobook.co.kr/images/book/large...,['이용' '분류' '딥러닝' '이해' '텍스트' '분석' '기반' '토픽' '기법...
1031,1031,"구글 BERT의 정석 :인공지능, 자연어 처리를 위한 BERT의 모든 것","수다르산 라비찬디란 지음;전희원,정승환,김형준 옮김",한빛미디어,9791162244852,004.735,2022-04-11,http://image.kyobobook.co.kr/images/book/large...,['모델' '언어' '이해' '학습' '처리' '트랜스포머' '보충' '자연어' '...
1050,1050,딥러닝 파이토치 교과서 :기본기에 충실한 딥러닝 입문서!,서지영 지음,길벗,9791165218942,004.73,2022-04-01,http://image.kyobobook.co.kr/images/book/large...,['딥러닝' '신경망' '파이토치' '학습' '구현' '알고리즘' '모델' '성능'...
1240,1240,파이토치로 배우는 자연어 처리,"델립 라오,브라이언 맥머핸 지음 ;박해선 옮김",한빛미디어,9791162244333,004.735,2021-06-28,http://image.kyobobook.co.kr/images/book/large...,['처리' '자연어' '딥러닝' '시퀀스' '예제' '학습' '모델' '신경망' '...
1566,1566,한국어 임베딩 =Sentence embendings using korean corp...,이기창 지음,에이콘출판,9791161753508,004.73,2019-10-08,http://image.kyobobook.co.kr/images/book/large...,['임베딩' '모델' '자연어' '학습' '단어' '한국어' '수준' '처리' '문...


In [ ]:

embedding_model.wv.similar_by_word('파이썬')